## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-05-16-54-04 +0000,nyt,Harm or Help? Why Companies Are Battling Tarif...,https://www.nytimes.com/2025/08/01/business/ec...
1,2025-08-05-16-53-34 +0000,nypost,Former X CEO Linda Yaccarino lands new gig at ...,https://nypost.com/2025/08/05/business/former-...
2,2025-08-05-16-47-54 +0000,nypost,Young adults are less likely to follow politic...,https://nypost.com/2025/08/05/us-news/young-ad...
3,2025-08-05-16-47-21 +0000,nyt,Tennessee Inmate Executed Despite Ethical Heal...,https://www.nytimes.com/2025/08/05/us/politics...
4,2025-08-05-16-42-24 +0000,nyt,Minimum Wage in L.A. Could Rise to $30 an Hour...,https://www.nytimes.com/2025/08/04/business/mi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
92,trump,51
13,new,19
50,house,17
232,gaza,17
233,war,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
217,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...,127
193,2025-08-05-01-00-00 +0000,wsj,President Trump’s high-stakes push to create m...,https://www.wsj.com/politics/texas-redistricti...,97
265,2025-08-04-20-30-20 +0000,nypost,Ex-Obama officials face federal grand jury pro...,https://nypost.com/2025/08/04/us-news/ex-obama...,90
21,2025-08-05-16-22-40 +0000,nypost,"Trump walks along White House roof, jokes he’s...",https://nypost.com/2025/08/05/us-news/trump-wa...,89
216,2025-08-04-22-55-43 +0000,latimes,Trump plans White House task force on security...,https://www.latimes.com/politics/story/2025-08...,87


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
217,127,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...
91,54,2025-08-05-13-27-00 +0000,cbc,"India, Russia hit back after U.S. accuses Modi...",https://www.cbc.ca/news/world/india-us-russia-...
89,53,2025-08-05-13-27-37 +0000,nypost,Jeffrey Epstein’s NYC den of depravity: Hidden...,https://nypost.com/2025/08/05/us-news/jeffrey-...
237,53,2025-08-04-21-48-14 +0000,nypost,Brazil’s Supreme Court orders house arrest for...,https://nypost.com/2025/08/04/world-news/brazi...
286,50,2025-08-04-18-44-19 +0000,nypost,Netanyahu approves plan to fully occupy Gaza S...,https://nypost.com/2025/08/04/world-news/netan...
86,45,2025-08-05-13-39-25 +0000,nypost,CEO ignored ‘red flags’ ahead of ‘preventable’...,https://nypost.com/2025/08/05/world-news/titan...
251,43,2025-08-04-21-04-00 +0000,wsj,The State Department may require travelers ent...,https://www.wsj.com/politics/policy/trump-admi...
112,41,2025-08-05-12-25-34 +0000,nypost,7-year-old girl survivor in deadly Miami boat ...,https://nypost.com/2025/08/05/us-news/7-year-o...
14,36,2025-08-05-16-34-00 +0000,wsj,The Swiss president is heading to the U.S. in ...,https://www.wsj.com/economy/trade/swiss-presid...
187,31,2025-08-05-01-17-08 +0000,latimes,Wealthy first-time candidate Cloobeck drops $1...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
